# LSST Verification Framework Demonstration

`lsst.verify` is the new framework for making verification measurements in the LSST Science Pipelines. *Verification* is an activity where well-defined quantities, called *metrics*, are measured to ensure that LSST's data and pipelines meet requirements, which we call *specifications*.

You might be familar with `validate_drp`. That package that currently measures metrics of `ProcessCcdTask` outputs and posts results to [SQUASH](https://squash.lsst.codes). By tracking metric measurements we are able to understand trends in the algorithmic performance of the LSST Science Pipelines, and ultimately verify that we will meet our requirements.

With `lsst.verify` we sought to generalize the process of defining metrics, measuring those metrics, and tracking those measurements in SQUASH. Rather than supporting only specially-design verification afterburner tasks, our goal is to empower developers to track performance metrics of their own specific pipeline Tasks. By defining metrics relevant to specific Tasks, verification becomes a highly relevant integration testing activity for day-to-day pipelines development.

This tutorial demonstrates key features and patterns in the `lsst.verify` framework, from defining metrics and specifications, to making measurements, to analyzing and summarizing performance.

In [1]:
import astropy.units as u
import yaml

import lsst.verify

## Defining metrics

Metrics are definitions of measurable things that you want to track. A measureable thing could be anything: the $\chi^2$ of a fit, the number of sources identified and measured, or even the latency or memory usage of a function.

In the verification framework, all metrics are centrally defined in the [verify_metrics](https://github.com/lsst/verify_metrics) package. To define a new metric, simply add or modify a YAML file in the `/metrics` directory of [verify_metrics](https://github.com/lsst/verify_metrics). Each Stack package that measures metrics has its own YAML definitions file (`jointcal.yaml`, `validate_drp.yaml`, and so on).

SQUASH watches `verify_metrics` so that when a metric is committed to the GitHub repo is it also known to the SQUASH dashboard.

For this tutorial, we will create metrics for hypothetical `demo1` and `demo2` packages. Here are some metric definitions.

First, content for a hypothetical `/metrics/demo1.yaml` file:

In [2]:
demo1_metrics_yaml = """
ZeropointRMS:
  unit: mmag
  description: >
    Photometric calibration RMS.
  reference:
    url: https://example.com/PhotRMS
  tags:
    - photometry

Complete:
  unit: mag
  description: >
    Magnitude of the catalog's 50% completeness limit.
  reference:
    url: https://example.com/Complete
  tags:
    - photometry

SourceCount:
  unit: ''
  description: >
    Number of objects in the output catalog.
  tags:
    - photometry
"""

This YAML defines three metrics: `demo1.ZeropointRMS`, `demo1.Complete` and `demo1.SourceCount`. A metric consists of:

- **A name.** Names are prefixed by name of the package that defines them.
- **A description.** This helps to document metrics, even if they are more thoroughly defined in other documentation (see the `reference` field).
- **A unit.** Units are `astropy.units`-compatibble strings. Metrics of unitless quantities should use the [dimensionless_unscaled}(http://docs.astropy.org/en/stable/units/standard_units.html#the-dimensionless-unit) unit, an empty string.
- **References.** References can be made to URLs, or even to document handles and page numbers. We can expand the `reference` field's schema to accomodate formalize reference identifiers in the future.
- **Tags.** These help us group metrics together in reports.

For the purposes of this demo, we'll parse this YAML object into a `lsst.verify.MetricSet` collection. Normally this doesn't need to be done since metrics should be pre-defined in `verify_metrics`, which are automatically loaded as we'll see later.

In [3]:
from tempfile import TemporaryDirectory
import os

with TemporaryDirectory() as temp_dir:
    demo1_metrics_path = os.path.join(temp_dir, 'demo1.yaml')
    with open(demo1_metrics_path, mode='w') as f:
        f.write(demo1_metrics_yaml)
    demo1_metrics = lsst.verify.MetricSet.load_single_package(demo1_metrics_path)
print(demo1_metrics)

<MetricSet: 3 Metrics>


## Defining metric specifications

Specifications are tests of metric measurements. A specification can be thought of as a milestone; if a measurement passes a specification then data and code are working as expected.

Like metrics, specifications can be centrally defined in the [verify_metrics](https://github.com/lsst/verify_metrics) repository. Specifications for each package are defined in one or more YAML files in the `specs` subdirectory of `verify_metrics`. See [`validate_drp`'s directory for an example](https://github.com/lsst/verify_metrics/tree/master/specs/validate_drp).

Here is a typical specification, in this case for the `demo1.ZeropointRMS` metric:

```yaml
name: minimum
metric: ZeropointRMS
threshold:
  operator: <=
  unit: mmag
  value: 20.0
tags:
  - demo-minimum
```

The fully-qualified name for this specification is `demo1.ZeropointRMS.minimum`, following a `{package}.{metric}.{spec_name}` format. Specifications names should be unique, but otherwise can be anything. The Verification Framework does not place special meaning on "minimum," "design," and "stretch" specifications. Instead, we recommend that you use tags to designate specifiations with operational meaning.

The core of a specification is its test, and the `demo1.ZeropointRMS.minimum` specification defines its test in the `threshold` YAML field. Here, a measurement *passes* the specification if $\mathrm{measurement} \leq 20.0~\mathrm{mmag}$.

We envision other types of specifications beyond thresholds (binary comparisions). Possible specification types include ranges and tolerances.

## Making specifications act only upon certain measurements

Often you'll make measurements of a metric in many contexts. With different datasets, from different cameras, in different filters. 